<a href="https://colab.research.google.com/github/srinath9121/vehicle-classification/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-be59bae1bcc8381f.parquet', 'validation': 'data/validation-00000-of-00001-58629e4f444d54b9.parquet', 'test': 'data/test-00000-of-00001-add65b65a0773a0f.parquet'}
df = pd.read_parquet("hf://datasets/james-burton/data_scientist_salary_all_text/" + splits["train"])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['salary'].isnull()

## Import libraries

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("DrBimmer/vehicle-classification")

In [ ]:
ds

In [ ]:
# View available splits (like train/test)
print(ds)

# Look at one sample
print(ds['train'][0])

# Convert to pandas DataFrame
import pandas as pd
df = pd.DataFrame(ds['train'])
print(df.head())


In [ ]:
!pip install -q datasets transformers accelerate torchvision


In [ ]:
print(ds)

In [ ]:
from transformers import AutoImageProcessor

model_checkpoint = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_checkpoint)

def preprocess(examples):
    images = [img.convert("RGB") for img in examples["image"]]
    inputs = processor(images, return_tensors="pt")
    inputs["labels"] = examples["type"]  # or "manufacturer"
    return inputs

# Preprocess the dataset
prepared_ds = ds.with_transform(preprocess)


In [ ]:
plt.imshow(ds['train'][0]['image'])


In [ ]:
labels = ds['train'].features['label']
print(labels)


In [ ]:
print(labels.int2str(0))
print(labels.int2str(1))


In [ ]:
import matplotlib.pyplot as plt

example = ds['train'][0]
plt.imshow(example['image'])
plt.title(f"Label: {ds['train'].features['label'].int2str(example['label'])}")
plt.axis('off')
plt.show()


# Load Pretrained Model

In [ ]:
# ✅ Define the pretrained model checkpoint
checkpoint = "google/vit-base-patch16-224"


In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForImageClassification.from_pretrained(checkpoint)


In [ ]:
num_labels = ds["train"].features["label"].num_classes
num_labels

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(checkpoint)

def preprocess_batch(examples):
    # examples is expected to be a dictionary of lists
    images = [img.convert("RGB") for img in examples["image"]]
    inputs = processor(images, return_tensors="pt")
    inputs["labels"] = examples["label"] # labels should be a list of integers
    return inputs

# Use map to preprocess the dataset upfront
prepared_ds = ds.map(preprocess_batch, batched=True)

# Select only the necessary columns
prepared_ds = prepared_ds.select_columns(['pixel_values', 'labels'])

In [ ]:
pip install --upgrade transformers


In [ ]:
from transformers import AutoModelForImageClassification

num_labels = ds["train"].features["label"].num_classes
id2label = {i: ds["train"].features["label"].int2str(i) for i in range(num_labels)}
label2id = {v: k for k, v in id2label.items()}

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
     ignore_mismatched_sizes=True

)

In [ ]:
!pip install -q evaluate

In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

training_args = TrainingArguments(
    output_dir="./vehicle-vit",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    push_to_hub=False
)
accuracy

In [ ]:
print(accuracy)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=processor,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
metrics = trainer.evaluate()
print("Validation Accuracy:", metrics["eval_accuracy"])


In [ ]:
import torch, random

sample = random.choice(ds["validation"])
inputs = processor(sample["image"].convert("RGB"), return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model(**inputs)
    pred_id = outputs.logits.argmax(-1).item()

pred_label = id2label[pred_id]

plt.imshow(sample["image"])
plt.title(f"Predicted: {pred_label}")
plt.axis("off")
plt.show()


In [ ]:
model.save_pretrained("./vehicle_model")
processor.save_pretrained("./vehicle_model")


In [ ]:
print(ds)

In [ ]:
checkpoint = "/content/my_finetuned_model"
data_dir = "/content/validation_data"


In [ ]:
data_dir

In [ ]:
# ================================
# 🔹 STEP 1: Imports
# ================================
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import classification_report
import os # Import os to check for directory existence

# ================================
# 🔹 STEP 2: Load model and processor
# Replace 'your_checkpoint' with your model name or saved folder
# ================================
model_path = "./vehicle_model" # Use the path to your saved model

# Check if the model path exists
if not os.path.exists(model_path):
    print(f"Error: Model directory not found at {model_path}. Please ensure the directory exists.")
else:
    model = AutoModelForImageClassification.from_pretrained(model_path)
    processor = AutoImageProcessor.from_pretrained(model_path)

    # ================================
    # 🔹 STEP 3: Load new validation images
    # Make sure your folder structure is:
    # validation_data/
    # ├── sedan/
    # ├── suv/
    # ├── truck/
    # ================================
    data_dir = "/content/validation_data"  # <-- change path to your folder

    # Check if the data directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Data directory not found at {data_dir}. Please ensure the directory exists and contains subfolders with images.")
    else:
        try:
            val_ds = load_dataset("imagefolder", data_dir=data_dir, split="train")
            print(val_ds)

            # ================================
            # 🔹 STEP 4: Preprocess function
            # This preprocesses a batch of examples
            # ================================
            def preprocess_batch(examples):
                # examples is expected to be a dictionary of lists
                images = [img.convert("RGB") for img in examples["image"]]
                inputs = processor(images, return_tensors="pt")
                inputs["labels"] = examples["label"]
                return inputs

            # Apply preprocessing as a transformation (efficient for large datasets)
            # This applies the preprocessing when data is accessed
            val_ds = val_ds.with_transform(preprocess_batch)

            # ================================
            # 🔹 STEP 5: Run Predictions
            # ================================
            model.eval()

            pred_labels = []
            true_labels = []

            # Iterate through the dataset using a DataLoader for batching
            from torch.utils.data import DataLoader

            # The collate_fn will be implicitly handled by with_transform if it returns tensors
            dataloader = DataLoader(val_ds, batch_size=16) # Adjust batch size as needed

            print("Running predictions...")
            for batch in dataloader:
                inputs = {k: v.to(model.device) for k, v in batch.items()}
                with torch.no_grad():
                    outputs = model(**inputs)

                logits = outputs.logits
                batch_pred_ids = logits.argmax(-1).tolist()
                pred_labels.extend(batch_pred_ids)
                true_labels.extend(inputs["labels"].tolist())


            # ================================
            # 🔹 STEP 6: Convert IDs to names
            # ================================
            # Ensure the model's id2label is used, which was loaded from the saved model
            id2label = model.config.id2label
            pred_names = [id2label[i] for i in pred_labels]
            true_names = [id2label[i] for i in true_labels]

            # ================================
            # 🔹 STEP 7: Accuracy + Report
            # ================================
            accuracy = np.mean(np.array(pred_labels) == np.array(true_labels))
            print(f"✅ Validation Accuracy: {accuracy:.2%}")

            print("\nClassification Report:")
            # Ensure that all labels present in true_names and pred_names are included in the report
            # This can be an issue if a class is not present in the validation set
            # Get all possible labels from the model's id2label
            target_names = list(id2label.values())

            print(classification_report(true_names, pred_names, target_names=target_names, zero_division=0))

            # ================================
            # 🔹 STEP 8: Visualize random predictions
            # Visualize a few samples from the original dataset for plotting
            # Need to reload or access original images for visualization
            # For simplicity, let's just show the predicted and true labels
            # To show images, you would need to access the original images, perhaps by
            # re-loading a few samples from the original imagefolder dataset or
            # storing the original images/paths during loading.

            print("\nExample Predictions (True | Predicted):")
            # Select a few random indices to display
            random_indices = random.sample(range(len(true_names)), min(5, len(true_names)))

            for i in random_indices:
                 print(f"True: {true_names[i]} | Predicted: {pred_names[i]}")

            # Note: To visualize the actual images with labels, you would need to
            # access the image data from the original dataset or store paths.
            # The val_ds with transform applied doesn't directly hold image objects anymore.


        except Exception as e:
            print(f"An error occurred during dataset loading or prediction: {e}")

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
# 🔹 Unzip your uploaded file into /content/validation_data(1).zip
!unzip -q validation_data.zip -d /content/validation_data(1).zip

# 🔹 Check whether images and subfolders are extracted properly
!ls /content/validation_data(1).zip


In [ ]:
from datasets import load_dataset

data_dir = "/content/validation_data"
val_ds = load_dataset("imagefolder", data_dir=data_dir, split="train")

print(val_ds)
